In [1]:
import json
import os
import httpx
import dotenv
from mcp.server.fastmcp import FastMCP
from loguru import logger

dotenv.load_dotenv()

# 创建FastMCP实例，用于启动天气服务器SSE服务
mcp = FastMCP("WeatherServerSSE", host="0.0.0.0", port=8000)



@mcp.tool()
def get_weather(city: str):
    """
    查询指定城市的即时天气信息。

    :param city: 必要参数，字符串类型，表示要查询天气的城市名称。
                 注意：中国城市需使用其英文名称，如 "Beijing" 表示北京。
    :return: 返回 OpenWeather API 的响应结果，URL 为
             https://api.openweathermap.org/data/2.5/weather。
             响应内容为 JSON 格式的字符串，包含详细的天气数据。
    """

    # 发送 GET 请求并获取响应
    # response = httpx.get(url, params=params)
    if city.lower() == "beijing":
        response = {
            "coord": {"lon": 116.4074, "lat": 39.9042},
            "weather": [{"id": 801, "main": "Clouds", "description": "few clouds", "icon": "02d"}],
            "base": "stations",
            "main": {"temp": 12.5, "feels_like": 11.9, "temp_min": 12.5, "temp_max": 12.5, "pressure": 1012,
                     "humidity": 52},
            "visibility": 6000,
            "wind": {"speed": 3.6, "deg": 80},
            "clouds": {"all": 23},
            "dt": 1671673146,
            "sys": {"type": 1, "id": 9246, "country": "CN", "sunrise": 1671650426, "sunset": 1671693807},
            "timezone": 28800,
            "id": 1816670,
            "name": "Beijing",
            "cod": 200
        }

    elif city.lower() == "shanghai":
        response = {
            "coord": {"lon": 121.4737, "lat": 31.2304},
            "weather": [{"id": 802, "main": "Clouds", "description": "scattered clouds", "icon": "03d"}],
            "base": "stations",
            "main": {"temp": 18.2, "feels_like": 17.8, "temp_min": 18.2, "temp_max": 18.2, "pressure": 1010, "humidity": 72},
            "visibility": 5000,
            "wind": {"speed": 4.1, "deg": 150},
            "clouds": {"all": 38},
            "dt": 1671673146,
            "sys": {"type": 1, "id": 9261, "country": "CN", "sunrise": 1671650044, "sunset": 1671692578},
            "timezone": 28800,
            "id": 1796236,
            "name": "Shanghai",
            "cod": 200
        }
    else:
        return {
            "error": "city_not_found",
            "message": "城市未找到，请输入有效城市名称"
        }

    # 将响应解析为 JSON 并序列化为字符串返回
    # data = response.json()
    # logger.info(f"查询天气结果：{json.dumps(data)}")
    logger.info(f"查询天气结果：{response}")
    return response

import  asyncio

async def run_mcp():
    mcp.run(transport="sse")

if __name__ == "__main__":
    logger.info("启动 MCP SSE 天气服务器，监听 http://0.0.0.0:8000/sse")
    # 运行MCP客户端，使用Server-Sent Events(SSE)作为传输协议
    mcp.run(transport="sse")
    # asyncio.run(run_mcp())
    

2025-12-17 11:28:17.857 | INFO     | __main__:<module>:81 - 启动 MCP SSE 天气服务器，监听 http://0.0.0.0:8000/sse


RuntimeError: Already running asyncio in this thread

In [2]:
import asyncio
import json
from typing import Any, Dict
from dotenv import load_dotenv
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from loguru import logger

# 加载 .env 文件中的环境变量，override=True 表示覆盖已存在的变量
load_dotenv(override=True)

checkpointer = InMemorySaver()
config = {"configurable": {"thread_id": "user-001"}}


def load_servers(file_path: str = "mcp.json") -> Dict[str, Any]:
    """
    从指定的 JSON 文件中加载 MCP 服务器配置。

    参数:
        file_path (str): 配置文件路径，默认为 "mcp.json"

    返回:
        Dict[str, Any]: 包含 MCP 服务器配置的字典，若文件中没有 "mcpServers" 键则返回空字典
    """
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
        return data.get("mcpServers", {})


async def run_chat_loop() -> None:
    """
    启动并运行一个基于 MCP 工具的聊天代理循环。

    该函数会：
    1. 加载 MCP 服务器配置；
    2. 初始化 MCP 客户端并获取工具；
    3. 创建基于 Ollama 的语言模型和代理；
    4. 启动命令行聊天循环；
    5. 在退出时清理资源。

    返回:
        None
    """
    # 1️ 加载服务器配置
    servers_cfg = load_servers()

    # 2️ 初始化 MCP 客户端并获取工具
    mcp_client = MultiServerMCPClient(servers_cfg)
    tools = await mcp_client.get_tools()
    logger.info(f"✅ 已加载 {len(tools)} 个 MCP 工具： {[t.name for t in tools]}")

    # 3 初始化语言模型
    llm = ChatOllama(
        # model="deepseek-r1:32b",
        model="llama3.1:8b",
        base_url="http://localhost:12356"
    )
    # 4 构建LangGraph Agent
    prompt = """
    你是一个智能体，可以调用以下函数：
    1. get_weather(city: str) —— 获取指定地点的天气
    2. fetch(url: str) —— 请求指定 URL 并返回内容网页的内容
    
    请根据用户的自然语言请求，判断是否需要调用函数，并严格按照函数输入格式返回调用指令。
    如果不需要调用函数，就直接回答。
    """
    agent = create_react_agent(model=llm, prompt=prompt, tools=tools, checkpointer=checkpointer)
    # 5. CLI聊天
    logger.info("\n🤖 MCP Agent 已启动，输入 'quit' 退出")
    while True:
        user_input = input("\n你: ").strip()
        if user_input.lower() == "quit":
            break
        try:
            result = await agent.ainvoke({"messages": [("user", user_input)]}, config)
            print(f"\nAI: {result['messages'][-1].content}")
        except Exception as exc:
            logger.error(f"\n⚠️  出错: {exc}")

    # 6. 退出会话
    logger.info("🧹 已退出会话，Bye!")


if __name__ == "__main__":
    # 启动异步事件循环并运行聊天代理
    asyncio.run(run_chat_loop())


RuntimeError: asyncio.run() cannot be called from a running event loop